# Test advection-diffusion model


In [ ]:
# imports
import numpy as np
import cuqi
import sys
import matplotlib.pyplot as plt
from cuqi.distribution import Gaussian, JointDistribution, GMRF, Gamma
from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
from copy import deepcopy
from custom_distribution import MyDistribution
from advection_diffusion_inference_utils import parse_commandline_args,\
    read_data_files,\
    create_domain_geometry,\
    create_PDE_form,\
    create_prior_distribution,\
    create_exact_solution_and_data,\
    set_the_noise_std,\
    sample_the_posterior,\
    create_experiment_tag,\
    plot_experiment,\
    save_experiment_data,\
    Args,\
    build_grids,\
    create_time_steps,\
    plot_time_series

#### Set up run arguments

In [ ]:
args = Args()
noise_level_list= ["fromDataVar" , "fromDataAvg", "avgOverTime", 0.1, 0.2]
args.noise_level = noise_level_list[1]
args.animal = 'm1'
args.ear = 'l'
args.num_ST = 0
args.inference_type =  'advection_diffusion' 
args.unknown_par_type ='custom_1'
#args.unknown_par_value = [100]
args.rbc = 'fromData' #'none'#'fromData'
#args.unknown_par_value = ['m1:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5@../../../Collab-BrainEfflux-Data/April_2x_2024_b']
args.version = 'v200824_temp'
Gibbs = False
tag = create_experiment_tag(args)
print(tag)



In [ ]:
print(dir(args))
args.data_type

### Read the data

In [ ]:
times, locations, real_data, real_std_data = read_data_files(args)
# The left boundary condition is given by the data  
real_bc_l = abs(real_data.reshape([len(locations), len(times)])[0,:])
if args.rbc == 'fromData':
    real_bc_r = abs(real_data.reshape([len(locations), len(times)])[-1,:])
else:
    real_bc_r = None

### Create the forward model

In [ ]:
#%% STEP 4: Create the PDE grid and coefficients grid
#----------------------------------------------------
# PDE and coefficients grids
L = locations[-1]*1.01
coarsening_factor = 5
n_grid_c = 20
grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)

#%% STEP 5: Create the PDE time steps array
#------------------------------------------
tau_max = 30*60 # Final time in sec
cfl = 4 # The cfl condition to have a stable solution
         # the method is implicit, we can choose relatively large time steps 
tau = create_time_steps(h, cfl, tau_max)

#%% STEP 6: Create the domain geometry
#-------------------------------------
G_c = create_domain_geometry(grid_c, args.inference_type)

# STEP 7: Create the PDE form
#----------------------------
PDE_form = create_PDE_form(real_bc_l, real_bc_r,
                           grid, grid_c, grid_c_fine, n_grid, h, times,
                           args.inference_type)

# STEP 8: Create the CUQIpy PDE object
#-------------------------------------
PDE = TimeDependentLinearPDE(PDE_form,
                             tau,
                             grid_sol=grid,
                             method='backward_euler', 
                             grid_obs=locations,
                             time_obs=times) 

# STEP 9: Create the range geometry
#----------------------------------
G_cont2D = Continuous2D((locations, times))

# STEP 10: Create the CUQIpy PDE model
#-------------------------------------
A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

In [ ]:
PDE.time_steps

#### Create the prior distribution

In [ ]:
# STEP 11: Create the prior distribution
#---------------------------------------
#x = create_prior_distribution(G_c, args.inference_type)
#x1 = GMRF(np.ones(G_c.par_dim-1)*np.sqrt(300),
#            0.2,
#            bc_type='neumann')
#x2 = Gaussian(0.5, 0.3**2)
#x = MyDistribution([x1, x2], geometry=G_c )
x = create_prior_distribution(G_c, args.inference_type)


#from cuqi.distribution import GMRF
#x = GMRF(np.ones(G_c.par_dim)*np.sqrt(300), 0.2, geometry=G_c, bc_type='neumann')

Ns = 10
samples_x = x.sample(Ns)
samples_x.plot(range(Ns), plot_par=True)
plt.figure()
samples_x.funvals.plot(range(Ns))

In [ ]:
# Plot prior CI
x.sample(1000).funvals.plot_ci(95)

In [ ]:
# Test different prior
x_alternative = cuqi.distribution.GMRF(np.ones(G_c.par_dim)*np.sqrt(100), 2, geometry=G_c, bc_type='neumann')
x_alternative.sample(1000).plot_ci(95)

#### Create and plot exact data

In [ ]:
sqrt_a =0.245 #np.sqrt(0.9)
x_true, exact_data = create_exact_solution_and_data(A, args.unknown_par_type, args.unknown_par_value, a=sqrt_a, grid_c=grid_c)
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]))

plt.figure()
plot_time_series(times, locations, real_data.reshape([len(locations), len(times)]))


In [ ]:
PDE.assemble(x_true)
sol, _ = PDE.solve()
print(sol.shape)

plt.plot(grid, sol)

len(tau)
plt.figure()
x_true.funvals.plot()


#### Create the data distribution

In [ ]:
#%% STEP 13: Create the data distribution
#----------------------------------------
# First, illustrate how different noise levels setups affect the std
print("Standard deviation values")
plt.figure()
for item in noise_level_list:
    s_noise_temp = set_the_noise_std(
        args.data_type, item, exact_data,
        None, real_std_data, G_cont2D)
    print('\n**noise_level option**:', item)
    print('**STD of the noise**:')
    print(s_noise_temp)
    # plot noise level
    if isinstance(s_noise_temp, np.ndarray):
        plt.plot(s_noise_temp, label=str(item))
    else:
        plt.plot(s_noise_temp*np.ones(G_cont2D.par_dim), label=str(item))
plt.legend()
plt.title('Noise level')
plt.xlabel('data point index i\n data point for the same location are grouped together')
plt.ylabel('std of the noise')

# Second, set the noise level
s_noise = set_the_noise_std(args.data_type, args.noise_level, exact_data,
                                None, real_std_data, G_cont2D)

print("The applied noise level is: ", s_noise)


#### Create the data distribution

In [ ]:

if Gibbs:
    s = Gamma(1, 500)
    s_samples = s.sample(1000)
    s_samples.plot_trace()
    s_true = 1/s_noise**2
    print(s_true)
    y_h = Gaussian(A(x), lambda s: 1/s, geometry=G_cont2D)
else:
    y = Gaussian(A(x), s_noise**2, geometry=G_cont2D)

#### Samples of noisy data and plot the noisy data and the noise

In [ ]:
if Gibbs:
    noisy_data_h = y_h(x=x_true, s=s_true).sample()

else:
    noisy_data = y(x=x_true).sample()

fig, ax = plt.subplots(1, 3, figsize=(14, 3))
plt.sca(ax[0])
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]), plot_legend=False)
plt.title('Exact data')

if Gibbs:
    plt.sca(ax[1])
    plot_time_series(times, locations, noisy_data_h.reshape([len(locations), len(times)]), plot_legend=False)
    plt.title('Noisy data (hyper prior)')
    diff_h = noisy_data_h - exact_data
    plt.sca(ax[2])
    lines, labels = plot_time_series(times, locations, diff_h.reshape([len(locations), len(times)]), plot_legend=False)
    plt.title('Difference (hyper prior)')
    # Noise to signal ratio
    print('Noise to signal ratio (synthitic data) (hyper prior): ', np.linalg.norm(diff_h)/np.linalg.norm(noisy_data_h))
else:

    plt.sca(ax[1])
    plot_time_series(times, locations, noisy_data.reshape([len(locations), len(times)]), plot_legend=False)
    plt.title('Noisy data')
    diff = noisy_data - exact_data
    plt.sca(ax[2])
    lines, labels = plot_time_series(times, locations, diff.reshape([len(locations), len(times)]), plot_legend=False)
    plt.title('Difference')
    # Noise to signal ratio
    print('Noise to signal ratio (synthitic data): ', np.linalg.norm(diff)/np.linalg.norm(noisy_data))






print('STD to signal ratio (real data): ', np.linalg.norm(real_std_data)/np.linalg.norm(real_data))

fig2 = plt.figure()
# turn off axis
plt.axis('off')
plt.legend(lines, labels, loc='center',ncol=3)
#figure size
fig2.set_size_inches(14, 1)



#### Create the Bayesian model and sample the posterior

In [ ]:
# Create the Bayesian model
#--------------------------
if not Gibbs:

    BP = cuqi.problem.BayesianProblem(x, y)
    BP.set_data(y=noisy_data)
    BP.posterior.enable_FD()
    x0 = np.ones(G_c.par_dim)*10
    x0[-1] = 1
    #map = BP.MAP(x0=x0)
    #BP.UQ(100)
    MH = cuqi.experimental.mcmc.MH(BP.posterior, initial_point=x0)
    MH.warmup(10)
    MH.sample(10)

if Gibbs:
    joint = cuqi.distribution.JointDistribution(x, y_h, s)
    joint_post = joint(y_h=noisy_data_h)
    sampling_strategy = {
        "x" : cuqi.experimental.mcmc.NUTSNew(max_depth=10),
        "s" : cuqi.experimental.mcmc.ConjugateNew()
    }
    
    sampler_h = cuqi.experimental.mcmc.HybridGibbsNew(joint_post, sampling_strategy)
    
    sampler_h.warmup(50)
    sampler_h.sample(200)
    samples_h = sampler_h.get_samples()
    
    samples_h["x"].plot_ci()
    

In [ ]:
if not Gibbs:
    samples = MH.get_samples()
    samples.plot_ci(exact=x_true)
    print(np.mean(MH._acc))
    print(samples.mean()[-1])
    print(x_true[-1])
    plt.figure()
    samples.plot_trace()
    samples.compute_ess()

if Gibbs:
    samples_h["s"].plot_ci(exact=s_true)
    plt.figure()
    samples_h["x"].plot_ci(95, exact=x_true)
    plt.figure()
    samples_h["s"].plot_trace()
    plt.figure()
    samples_h["x"].plot_trace()
    print(s_true)
    #samples_h["s"].plot_ci(95, exact=s_true)
    print(samples_h["s"].burnthin(150).mean())
    
    inferred_s_noise = 1/np.sqrt(samples_h["s"].burnthin(150).mean())
    print(s_noise)
    print(inferred_s_noise)

Demo forward with a = 1 , a = 2, a = 3 (instability in the last case)

In [ ]:
map = BP.MAP(x0=x0)

In [ ]:
map.plot(label='MAP')
x_true.plot(label='True solution', marker='x')
plt.plot(x0, 'o', label='Initial guess')
mean = np.concatenate((x.distribution_list[0].mean.flatten(), x.distribution_list[1].mean))
plt.plot(mean, '.', label='Prior mean')
plt.legend(loc='lower center')

In [ ]:
min(x_true[:-1])**2
max(x_true[:-1])**2
print(x_true[-1])
print(map[-1])


a = Pec * min_diffusion / L
Pec_min = a * L / min_diffusion
Pec_max = a * L / max_diffusion
Pec_min = 0.9**2 * 1900 / 21 = 72.9?
Pec_max = 0.9**2 * 1900 / 200 = 8.19?


a = 0.9 micro m/s

L = ~1900 micro m

min_diffusion = 21 micro m^2/s

http://calliope.dem.uniud.it/CLASS/ING-AMB/ade.pdf

0.01 to get pec of 0.1

0.01 * 2000 / (15)**2 = 20 /225 = 0.0889
a = sqrt(0.01) = 0.1

CA only inference 


In [ ]:
2/(0.3*0.01)

In [ ]:
np.sqrt(3000)

In [ ]:
import numpy as np
max_diff = x_true[:-1].max()**2
print("max_diff", max_diff)
min_diff = x_true[:-1].min()**2
print("min_diff", min_diff) 
sqrt_a2 = np.sqrt(0.06)
a2 = sqrt_a2**2
L = 400
Pec = a2*L/max_diff
print("Pec", Pec)
print("a", a2)



In [ ]:
np.sqrt(a2)

In [ ]:
pec_std= 1
a2_std = pec_std*max_diff/L
print("a_std", a2_std)
print("a_std_sqrt", np.sqrt(a2_std))

### A look into the prior distribution of the advection

1. Extract the prior distribution of the advection

In [ ]:

prior_a = deepcopy(x.distribution_list[1])

print(prior_a)
print(prior_a.geometry)
prior_a.sample(10000).funvals.plot_trace()

2. Equip it with the right geometry

In [ ]:
geom = cuqi.geometry.MappedGeometry(cuqi.geometry.Discrete(1), map = x.geometry.map)
prior_a.geometry = geom
prior_a_samples = prior_a.sample(10000)
prior_a_samples.funvals.plot_trace()
print("mean", prior_a_samples.funvals.mean())
print("std", prior_a_samples.funvals.std())


3. Now we look at Peclet number distribution

In [ ]:
pec_prior_max = deepcopy(prior_a)
pec_prior_geometry_max = cuqi.geometry.MappedGeometry(
    cuqi.geometry.Discrete(1),
    map = lambda val: val**2*L/min_diff)
pec_prior_max.geometry = pec_prior_geometry_max
pec_prior_max_samples = pec_prior_max.sample(10000)
pec_prior_max_samples.funvals.plot_trace()
print("max peclet")
print("L", L)
print("min_diffusion", min_diff)
print("pec_prior_max mean", pec_prior_max_samples.funvals.mean())
print("pec_prior_max std", pec_prior_max_samples.funvals.std())





In [ ]:
pec_prior_min = deepcopy(prior_a)
pec_prior_geometry_min = cuqi.geometry.MappedGeometry(
    cuqi.geometry.Discrete(1),
    map = lambda val: val**2*L/max_diff)
pec_prior_min.geometry = pec_prior_geometry_min
pec_prior_min_samples = pec_prior_min.sample(10000)
pec_prior_min_samples.funvals.plot_trace()
print("min peclet")
print("L", L)
print("max_diff", max_diff)
print("pec_prior_min mean", pec_prior_min_samples.funvals.mean())
print("pec_prior_min std", pec_prior_min_samples.funvals.std())

4. Function of peclet number given the advection speed


In [ ]:
map_adv_pec_max = lambda a: a * L / min_diff
a_grid = np.linspace(0, 1, 100)
plt.plot(a_grid, map_adv_pec_max(a_grid))
plt.xlabel('a')
plt.ylabel('Peclet number')
plt.title('Peclet number as a function of a (min diffusivity)')

In [ ]:
map_adv_pec_min = lambda a: a * L / max_diff
a_grid = np.linspace(0, 1, 100)
plt.plot(a_grid, map_adv_pec_min(a_grid))
plt.xlabel('a')
plt.ylabel('Peclet number')
plt.title('Peclet number as a function of a (max diffusivity)')
plt.show()
